In [1]:
import os
os.chdir('..')

In [5]:
import pandas as pd

from src.construct import util

idx = pd.IndexSlice

In [3]:
annual_energy_balance = util.read_tdf('build/annual_energy_balances.csv')

In [30]:
year = 2016
other_energy = annual_energy_balance.loc[idx[['FC_OTH_AF_E', 'FC_OTH_FISH_E', 'FC_OTH_NSP_E'], :, 'TJ', :, :]]

In [32]:
other_energy_fuels = other_energy.loc[idx[:, ['C0000X0350-0370', 'O4000XBIO', 'G3000', 'RA000', 'W6100_6220', 'H8000'], :, :, year]]

In [33]:
other_energy_fuels.sum(level=['cat_code', 'carrier_code']).unstack()

carrier_code,C0000X0350-0370,G3000,H8000,O4000XBIO,RA000,W6100_6220
cat_code,,,,,,
FC_OTH_AF_E,43018.088,158319.597,11122.292,581115.237,119341.045,35.0
FC_OTH_FISH_E,0.000,48.812,1474.700,69491.858,2924.047,0.0
FC_OTH_NSP_E,1660.167,35913.550,6011.851,82109.109,60565.692,0.0


In [36]:
other_energy.loc[idx[['FC_OTH_AF_E', 'FC_OTH_FISH_E', 'FC_OTH_NSP_E'], ['C0000X0350-0370', 'G3000', 'RA000', 'W6100_6220', 'H8000'], :, :, :]]

cat_code      carrier_code     unit  country  year  
FC_OTH_AF_E   C0000X0350-0370  TJ    AL       1990.0    0.0
                                              1991.0    0.0
                                              1992.0    0.0
                                              1993.0    0.0
                                              1994.0    0.0
                                                       ... 
FC_OTH_NSP_E  W6100_6220       TJ    UK       2014.0    0.0
                                              2015.0    0.0
                                              2016.0    0.0
                                              2017.0    0.0
                                              2018.0    0.0
Name: 0, Length: 14205, dtype: float64

In [88]:
carrier_names_df = pd.read_csv('data/energy_balance_carrier_names.csv', index_col=0, header=0)
balance_code = 'oth'
country_codes = {
        i: util.get_alpha3(i) for i in annual_energy_balance.index.get_level_values('country').unique()
    }
annual_energy_balance.loc[['FC_OTH_AF_E', 'FC_OTH_FISH_E', 'FC_OTH_NSP_E']].xs('TJ', level='unit').apply(util.tj_to_twh).unstack('year').groupby(
                [carrier_names_df[f'{balance_code}_carrier_name'].dropna().to_dict(), country_codes],
                level=['carrier_code', 'country']
            ).sum().rename_axis(['carrier_name', 'country_code'], axis=0).sum(level='carrier_name')

year,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,...,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
carrier_name,,,,,,,,,,,,,,,,,,,,,
ambient_heat,0.011667,0.011389,0.012222,0.013056,0.013611,0.014167,0.014722,0.015556,0.016389,0.016944,...,0.127272,0.136805,12.252376,12.745948,13.258128,13.064488,13.115267,15.277671,15.416773,16.237124
biofuel,21.024806,21.236417,20.581083,16.861306,16.186333,16.205528,15.984639,15.682139,15.234500,15.465417,...,20.562651,22.346836,19.735773,20.443988,21.337307,21.955028,20.277586,20.918876,21.144970,19.801697
gas,139.341136,158.026616,121.801516,126.851560,110.424439,117.647750,140.454311,131.562365,129.207406,128.483336,...,64.025730,78.916013,73.742762,74.902406,75.395499,70.843942,63.842062,64.440376,64.000974,60.751810
heat,31.493333,33.615833,40.888611,35.829444,32.030278,34.980556,41.036111,32.389444,32.561944,32.150000,...,5.753614,5.272012,4.325896,4.538072,4.374532,5.576782,4.970719,5.169123,4.733960,4.323800
oil,0.006161,0.006842,0.007584,0.026218,0.069133,0.011051,0.011986,0.015536,0.013989,0.015392,...,1.303906,1.109597,0.759171,0.935342,1.085510,1.211502,1.324092,1.501760,1.730798,3.011452
renewable_heat,2.729448,2.753146,2.938960,3.027023,3.060634,3.454712,3.446474,3.480820,3.542051,3.528815,...,2.231369,1.544019,1.711061,1.723810,1.736223,2.180394,2.363206,2.624578,2.739519,2.948468
solid_fossil,43.758950,45.386346,36.322564,40.232045,34.585149,29.078817,25.074527,23.307037,20.241672,21.949206,...,15.187431,16.608547,14.450404,14.802296,14.343481,13.495664,12.289181,13.093691,13.934666,13.579621


In [68]:
annual_energy_balance[
            annual_energy_balance.index.get_level_values('cat_code').str.contains(r'^FC_OTH_HH_E')
        ]

cat_code     carrier_code  unit  country  year  
FC_OTH_HH_E  BIOE          TJ    AL       1990.0    15199.0
                                          1991.0    15199.0
                                          1992.0    15199.0
                                          1993.0    14507.0
                                          1994.0    13847.0
                                                     ...   
             W6220         TJ    UK       2014.0      295.0
                                          2015.0      248.0
                                          2016.0        0.0
                                          2017.0        0.0
                                          2018.0        0.0
Name: 0, Length: 59971, dtype: float64

In [64]:
annual_energy_balance.loc[['FC_TRA_DAVI_E', 'INTAVI']].unstack('carrier_code')['O4000XBIO'].sum(level=['country', 'year']).sub(other_energy.loc[idx['FC_OTH_NSP_E', ['O4651', 'O4653', 'O4661XR5230B', 'O4669'], 'TJ', :, :]].sum(level=['country', 'year']), fill_value=0).apply(util.tj_to_twh).sum()

15877.024468055555

In [40]:
# assumption: agriculture oil use goes to heavy-duty transport demand; fisheries' oil use goes to marine fuel demand; others oil use goes predominantly to heavy-duty transport, except kerosene which goes to aviation
# electricity goes to everything else (solid fossils, nat gas, biofuels, direct heat, waste) goes to heat demand

heat_demand_approx = x
aviation_demand_approx = other_energy.loc[idx['FC_OTH_NSP_E', ['O4651', 'O4653', 'O4661XR5230B', 'O4669'], 'TJ', :, :]].sum(level=['country', 'year'])
hdv_demand_approx = other_energy.loc[idx[['FC_OTH_AF_E', 'FC_OTH_NSP_E'], 'O4000XBIO', 'TJ', :, :]].sum(level=['country', 'year']) - aviation_demand_approx
marine_demand_approx = other_energy.loc[idx[['FC_OTH_FISH_E'], 'O4000XBIO', 'TJ', :, :]].sum(level=['country', 'year'])

In [44]:
marine_demand_approx.xs('UK')

year
1990.0       0.000
1991.0       0.000
1992.0       0.000
1993.0       0.000
1994.0       0.000
1995.0       0.000
1996.0       0.000
1997.0       0.000
1998.0       0.000
1999.0       0.000
2000.0       0.000
2001.0       0.000
2002.0       0.000
2003.0       0.000
2004.0       0.000
2005.0       0.000
2006.0       0.000
2007.0       0.000
2008.0       0.000
2009.0       0.000
2010.0    3786.683
2011.0    3541.942
2012.0    3355.794
2013.0    3299.835
2014.0    3171.788
2015.0    5987.331
2016.0    6052.215
2017.0    3331.950
2018.0    6205.245
Name: 0, dtype: float64

In [20]:
util.ktoe_to_twh(69.4+406.8)

5.538206000000001

In [21]:
util.tj_to_twh(hdv_demand_approx.sum(level='country').xs('UK'))

13.953249999999999

In [22]:
util.ktoe_to_twh(190.9+9.5)

2.330652

In [27]:
other_energy.loc[idx[['FC_OTH_AF_E', 'FC_OTH_FISH_E', 'FC_OTH_NSP_E'], ['O4651', 'O4653', 'O4661XR5230B', 'O4669'], :, :, year]].sum(level=['cat_code'])

cat_code
FC_OTH_AF_E      15751.602
FC_OTH_FISH_E        0.000
FC_OTH_NSP_E     39498.351
Name: 0, dtype: float64